In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 5

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dropout (Dropout)                    │ (None, 42)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 20)                  │             860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │              55 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,125 (4.39 KB)

 Trainable params: 1,125 (4.39 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Model checkpoint callback
model_save_path = "model/keypoint_classifier/keypoint_classifier.keras"
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
import numpy as np

# Assuming 'y_true' is your current target labels array
num_classes = 5
y_true = np.array([5, 2, 0, 2, 2, 0, 0, 2, 2, 4, 3, 1, 2, 3, 0, 2, 2, 4, 2, 0, 3, 0, 1, 4, 4, 1, 5, 1, 1, 3, 0, 1, 5, 1, 3, 0, 0, 1, 3, 4, 1, 0, 0, 0, 3, 1, 2, 4, 0, 5, 0, 0, 5, 0, 1, 5, 1, 0, 0, 3, 1, 1, 0, 4, 0, 0, 0, 2, 0, 1, 1, 4, 2, 4, 2, 0, 2, 2, 2, 3, 2, 4, 2, 2, 1, 2, 1, 1, 4, 1, 0, 0, 2, 0, 1, 2, 4, 4, 0, 2, 1, 1, 0, 2, 0, 1, 5, 1, 0, 0, 1, 5, 1, 1, 2, 4, 0, 0, 3, 0, 3, 1, 0, 0, 5, 4, 4, 2])
y_true_corrected = np.remainder(y_true, num_classes)  # Correcting the labels to be within the valid range

In [ ]:
import tensorflow as tf
import numpy as np

# Assuming cp_callback and es_callback are properly defined
# Assuming X_train, y_train, X_test, y_test are loaded with data

try:
    # Define model architecture
    num_features = X_train.shape[1]  # Assuming X_train is your training data
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(num_features,)),  # Use the number of features in your input data
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Model training
    model.fit(
        X_train,
        y_train,
        epochs=1000,
        batch_size=128,
        validation_data=(X_test, y_test),
        callbacks=[cp_callback, es_callback]
    )

except Exception as e:
    print("An error occurred during model training:", e)


Epoch 1/1000


C:\Users\prasa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 1/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.0078 - loss: 2.6215
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0202 - loss: 2.4930 - val_accuracy: 0.3477 - val_loss: 2.0475
Epoch 2/1000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3594 - loss: 2.0296
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5380 - loss: 1.9029 - val_accuracy: 0.6270 - val_loss: 1.5596
Epoch 3/1000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6562 - loss: 1.5382
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6741 - loss: 1.4293 - val_accuracy: 0.6445 - val_loss: 1.2142
Epoch 4/1000
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6406 - loss: 1.2543
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.keras
12/12 ━━━━━

In [ ]:
import tensorflow as tf

try:
    # Model evaluation
    val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)
except tf.errors.InvalidArgumentError as e:
    print("An error occurred during model evaluation:", e)
    # Handle the error gracefully, e.g., log the error message or take corrective action
except Exception as e:
    print("An unexpected error occurred:", e)
    # Handle any other unexpected errors


In [ ]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [ ]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

# Confusion matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

# Convert to model for Tensorflow-Lite

In [ ]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)


In [ ]:
# Transform model (quantization)
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

# Inference test

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))